In [1]:
import re
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from netCDF4 import Dataset
import geopandas as gpd
from shapely.geometry import Point, box
import pandas as pd
import calendar
from datetime import datetime, timedelta
import seaborn as sns


from tqdm import tqdm
import rasterio
import rioxarray
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import mapping

In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

## Crop Planning Area

In [3]:
# Correct Factor for N Requirement of Crops in China
# S --> Planned Area of all crops [Major Only] --- Unit: 1000 hectares
# S_Total --> Planned Area of all crops [Major + Minor] --- Unit: 1000 hectares
# S_CropType --> Planned Area of each major croptype --- Unit: 1000 hectares
# N_Total --> N Requirement of all major crops --- 10000 tons
# R_CropType --> N Requirement of each major croptype --- Unit: Kg.N / ha

S_Dict = {'2017': 154012, '2018': 153354, '2019': 152753}
S_Total_Dict = {'2017':166332, '2018':165902, '2019':163931}

S_crop_data_2017 = {"Rice": 30747, "Wheat": 24508, "Corn": 42399, "Beans": 10051, "Tubers": 7173,\
                  "Oil-Bearing Crops": 13223, "Cotton": 3195, "Fiber Crops": 58, "Sugar Crops": 1546,\
                  "Tobacco": 1131, "Vegetables": 19981,}
S_crop_data_2018 = {"Rice": 30189, "Wheat": 24266, "Corn": 42130, "Beans": 10186, "Tubers": 7180,\
                  "Oil-Bearing Crops": 12872, "Cotton": 3354, "Fiber Crops": 57, "Sugar Crops": 1623, \
                  "Tobacco": 1058, "Vegetables": 20439}

S_crop_data_2019 = {"Rice": 29694, "Wheat": 23728, "Corn": 41284, "Beans": 11075, "Tubers": 7142,\
                  "Oil-Bearing Crops": 12925, "Cotton": 3339, "Fiber Crops": 66, "Sugar Crops": 1610,\
                  "Tobacco": 1027, "Vegetables": 20863}
S_CropType_Dict = {'2017': S_crop_data_2017, '2018': S_crop_data_2018, '2019': S_crop_data_2019,}

N_Total_Dict = {'2017':2221.8, '2018':2065.4, '2019':1930.2}

# Below is the Variable Info From the Zhang and Zhang (2012): UNIT is [kg/hectare]
R_CropType_Dict = {"Rice": 200, "Wheat": 219,"Corn": 203, 'Beans': 55, "Tubers": 226, \
                   'Oil-Bearing Crops': 128, "Cotton": 341, "Sugar Crops": 360, \
                   "Tobacco": 140, "Vegetables": 335}

## Decide types of crops planned in each grid of GFSAD1000

In [4]:
# Open GFSAD Data with China.shape

def mask_gfsad_with_shapefile(gfsad_path, shapefile_path):
    # Load the GFSAD data
    gfsad_data = rioxarray.open_rasterio(gfsad_path)

    # Load the shapefile
    china_shape = gpd.read_file(shapefile_path)

    # Set the CRS for the shapefile to EPSG:4326 if it is not set
    if china_shape.crs is None:
        china_shape.set_crs(epsg=4326, inplace=True)

    # Transform the shapefile CRS to match the GFSAD data CRS if they are different
    if gfsad_data.rio.crs != china_shape.crs:
        china_shape = china_shape.to_crs(gfsad_data.rio.crs)

    # Clip the GFSAD data using the shapefile
    gfsad_clipped = gfsad_data.rio.clip(china_shape.geometry.apply(mapping), china_shape.crs, drop=True)

    return gfsad_clipped

gfsad_path = '/global/scratch/users/liuwenjin021011/data/GFSAD1000/GFSAD1KCD.2010.001.2016348142525.tif'
shapefile_path = '/global/home/users/liuwenjin021011/logs/fall_2023/China-Doundary-Shape.shp'

# GFSAD_df = mask_gfsad_with_shapefile(gfsad_path, shapefile_path)[0] # this is only 1 band


In [5]:
# decide types of crop planned in each grid based on layer number: https://lpdaac.usgs.gov/products/gfsad1kcdv001

GFSAD_Crop_Dict = {1: ['Wheat', 'Rice'], 
                   2: ['Wheat', 'Rice', 'Barley', 'Soybeans'], \
                   3: ['Wheat', 'Rice', 'Cotton', 'Orchards'],\
                   4: ['Wheat', 'Rice', 'Soybeans', 'Surgarcane', 'Corn', 'Cassava'],\
                   5: ['Wheat', 'Barley'],\
                   6: ['Corn', 'Soybeans'],\
                   7: ['Wheat', 'Corn', 'Rice', 'Barley', 'Soybeans'],\
                   8: ['Wheat', 'Maize', 'Rice', 'Barley', 'Soybeans']
                  }

# GFSAD_df[i][j] == layer_num is True, then GFSAD_df[i][j] has the corresponding planned crops
def map_gfsad_to_crops(gfsad_df, crop_dict):
    # Initialize an empty DataFrame to store the results
    df = pd.DataFrame(columns=['x', 'y', 'planned_crops'])

    # Iterate through each key-value pair in the GFSAD_Crop_Dict
    for value, crops in crop_dict.items():
        # Find the mask where GFSAD_df equals the current value
        mask = gfsad_df == value

        # Convert the mask to DataFrame
        masked_df = mask.to_dataframe(name='mask')

        # Filter out where mask is True
        filtered_df = masked_df[masked_df['mask']]

        # Create a column for planned crops
        filtered_df['planned_crops'] = [crops for _ in range(len(filtered_df))]

        # Append the result to the main DataFrame
        df = pd.concat([df, filtered_df.reset_index()[['x', 'y', 'planned_crops']]], ignore_index=True)

    return df

# GFSAD_df = map_gfsad_to_crops(GFSAD_df, GFSAD_Crop_Dict)

## Regrid GFSAD Grid to Match that of CHIRPS_SMAP

In [6]:
# Over Sample GFSAD_df to Match the grid of CHIRPS_SMAP, and perserve the content of 'planned crops'
# E.g.: if GFSAD_df[i][j] = ['A'], GFSAD_df[i+1][j] = ['B'], GFSAD_df[i][j+ 1] = ['C'], GFSAD_df[i+1][j+1] = ['A'],
# and after regriding, [i][j], [i+1][j], [i][j+1], [i+1][j+1] match to a new grid, 
# then the 'planned_crops' for the new grid should be ['A', 'B', 'C']

def get_grid_resolution(df, lat_col='lat', lon_col='lon'):
    # Calculate differences in latitudes and longitudes
    lat_diff = np.diff(sorted(df[lat_col].unique()))
    lon_diff = np.diff(sorted(df[lon_col].unique()))

    # Average resolution
    lat_res = np.mean(lat_diff)
    lon_res = np.mean(lon_diff)

    return lat_res, lon_res

# chirps_smap_res = get_grid_resolution(CHIRPS_SMAP_df)

# GFSAD_df.rename(columns={'y': 'lat', 'x': 'lon'}, inplace=True)
# gfsad_res = get_grid_resolution(GFSAD_df)

# print("CHIRPS_SMAP Resolution:", chirps_smap_res)
# print("GFSAD Resolution:", gfsad_res)

In [7]:
import tqdm
def regrid_gfsad_to_match_chirps_smap(GFSAD_gdf, CHIRPS_SMAP_df):
    # Define the grid size for CHIRPS_SMAP
    delta_lon, delta_lat = 0.08829211718930399, 0.09336100444534785

    # Convert CHIRPS_SMAP_df to GeoDataFrame with progress tracking
    geometries = [box(lon - delta_lon/2, lat - delta_lat/2, lon + delta_lon/2, lat + delta_lat/2) 
                  for lon, lat in zip(CHIRPS_SMAP_df['lon'], CHIRPS_SMAP_df['lat'])]
    CHIRPS_SMAP_gdf = gpd.GeoDataFrame(CHIRPS_SMAP_df, geometry=geometries, crs="EPSG:4326")
    
    # Perform spatial join with progress tracking
    tqdm.tqdm.pandas(desc="Spatial Join Progress")
    joined_gdf = gpd.sjoin(CHIRPS_SMAP_gdf, GFSAD_gdf, how='left', op='intersects')

    # Ensure all elements in 'planned_crops' are iterables (replace NaNs with empty lists)
    joined_gdf['planned_crops'] = joined_gdf['planned_crops'].apply(lambda x: x if isinstance(x, list) else [])

    # Group by CHIRPS_SMAP grid and aggregate 'planned_crops'
    tqdm.tqdm.pandas(desc="Aggregating Progress")
    regridded_gdf = joined_gdf.groupby(['lon_left', 'lat_left']).progress_apply(lambda x: list(set.union(*map(set, x['planned_crops'])))).reset_index()

    # Rename columns
    regridded_gdf.rename(columns={'lon_left': 'lon', 'lat_left': 'lat'}, inplace=True)

    return regridded_gdf

def regrid_in_chunks(GFSAD_gdf, CHIRPS_SMAP_df, chunk_size):
    
    # Initialize an empty DataFrame for the regridded data
    regridded_df = pd.DataFrame(columns=['lon', 'lat', 'planned_crops'])

    # Split the CHIRPS_SMAP_df into chunks
    num_chunks = len(CHIRPS_SMAP_df) // chunk_size + 1
    for i in range(num_chunks):
        print(f"Processing chunk {i+1}/{num_chunks}...")
        sub_df = CHIRPS_SMAP_df.iloc[i * chunk_size: (i+1) * chunk_size]

        # Perform spatial join and aggregation for each chunk
        chunk_regridded = regrid_gfsad_to_match_chirps_smap(GFSAD_gdf, sub_df)
        chunk_regridded.to_csv(f'/global/scratch/users/liuwenjin021011/data/GFSAD1000/Regridded_GFSAD_Chunk{i}_Year_2017_With_Status.csv', index = False, header = True)
        # Append the results to the main DataFrame
        regridded_df = pd.concat([regridded_df, chunk_regridded])
        
    
    return regridded_df


## Join Re-Gridded GFSAD data to CHIRPS-SMAP data

merged_df.to_csv('/global/scratch/users/liuwenjin021011/data/FactorA_CHIRPS_SMAP_GFSAD_Year_2019_With_Status.csv', index= False, header = True)


## Find SAI Value each grid 


### Dictionary Keys Mapping
#### Make Sure Planning Area, SAI Table, GFSAD Data have same set of keys/CropTypes

In [9]:
GFSAD_crops = ['Barley', 'Cassava', 'Corn', 'Cotton', 'Maize', 'Orchards', 'Rice', 'Soybeans', 'Surgarcane', 'Wheat']

SAI_Dict = {}
SAI_Dict['Rice'] = 22.48 / 100
SAI_Dict['Wheat'] = 0.47 / 100
SAI_Dict['Corn'] = 32.3 / 100
SAI_Dict['Soybean'] = 1.56 / 100
SAI_Dict['Cotton'] = 2.29 / 100
SAI_Dict['Rapeseed'] = 0.4 / 100
SAI_Dict['Peanut'] = 0.12 / 100
SAI_Dict['Cassava'] = 1.69 / 100
SAI_Dict['Sugarcane'] = 18.72 / 100
SAI_crops = list(SAI_Dict.keys())

S_crops = list(S_CropType_Dict['2019'].keys())

In [10]:
# S_CropType --> Planned Area of each major croptype --- Unit: 1000 hectares
# Modify SAI Dict to Match the keys of S_CropType

def map_SAI_to_S_crop_types(SAI_Dict, S_CropType_Dict):

    crop_mapping = {
        'Rice': 'Rice',
        'Wheat': 'Wheat',
        'Corn': 'Corn',
        'Soybean': 'Beans',  
        'Cassava' : 'Tubers',
        'Peanut' : 'Oil-Bearing Crops',
        'Rapeseed' : 'Oil-Bearing Crops',
        'Cotton': 'Cotton',
        'Sugarcane': 'Sugar Crops'
       
    }

    # Initialize new dictionary with new keys and zero values
    new_dict = {key: 0 for key in S_CropType_Dict}

    # Iterate and sum the SAI for each new crop key
    for old_key, value in SAI_Dict.items():
        new_key = crop_mapping.get(old_key)
        if new_key and new_key in new_dict:
        
            new_dict[new_key] += value

    return new_dict

# Notice that S_CropType_Dict keys are same for year 2017, 2018, 2019
# Mapped_SAI_Dict = map_SAI_to_S_crop_types(SAI_Dict, S_CropType_Dict['2017'])
# Mapped_SAI_Dict.keys()

#### Crop Planning Month
#### the SAI for each crop will not be calculated if crop is not planned in that month

In [11]:
crop_growing_seasons = {
    'Rice': [4, 5, 6, 7, 8, 9, 10],
    'Wheat': [10, 11, 12, 1, 2, 3, 4, 5],
    'Corn': [4, 5, 6, 7, 8, 9, 10],
    'Beans': [4, 5, 6, 7, 8, 9],
    'Tubers': [4, 5, 6, 7, 8, 9, 10],
    'Oil-Bearing Crops': [4, 5, 6, 7, 8, 9],
    'Cotton': [4, 5, 6, 7, 8, 9, 10],
    'Fiber Crops': [4, 5, 6, 7, 8],
    'Sugar Crops': [3, 4, 5, 6, 7, 8, 9, 10, 11],
    'Tobacco': [4, 5, 6, 7, 8, 9],
    'Vegetables': [3, 4, 5, 6, 7, 8, 9, 10, 11]
}

In [12]:
from tqdm import tqdm

def calculate_SAI(df, mapped_area_dict, n_requirement_dict, crop_mapping):
    
    # Calculate SAI
    SAIs = []
        
    # Iterate through each row in the dataframe with tqdm progress bar
    for index in tqdm(df.index, desc="Calculating SAI", unit="row"):
        row = df.loc[index]
        all_crops = row['planned crops']
        if len(all_crops) == 0:
            SAIs.append(0)
            continue
        # print('all crops:', all_crops)

        # Find Unique Crops to avoid doubt counting
        unique_crops = []
        for crop in all_crops:
            mapped_crop = crop_mapping.get(crop, crop)
            if mapped_crop not in unique_crops:
                unique_crops += [mapped_crop]       
        # print('unique crops after mapping:', unique_crops)
        
        # Do not include the Crop is the Crop is not planned in the corresponding Month
        unique_planned_crops = []
        month = datetime.strptime(row['time'], "%Y-%m-%d").month
        for crop in unique_crops:
            mapped_crop = crop_mapping.get(crop, crop)
            # print(month, crop, mapped_crop)
            if crop_growing_seasons.get(mapped_crop):
                if month in crop_growing_seasons.get(mapped_crop):
                    unique_planned_crops += [mapped_crop]
            
        # print('unique crops growing in the month', month, ' are ', unique_planned_crops)
        
        # Calculate total planned area for all crops in this grid
        planned_area = {}
        for mapped_crop in unique_planned_crops:
            planned_area[mapped_crop] = mapped_area_dict.get(mapped_crop, 0)
        total_area = sum(planned_area.values())
        # print(planned_area, total_area)
        

        total_pro = 0
        if total_area > 0:
            SAI = 0
            for mapped_crop in unique_planned_crops:
                crop_area = mapped_area_dict.get(mapped_crop, 0)
                crop_proportion = crop_area / total_area
                total_pro += crop_proportion

                crop_SAI = Mapped_SAI_Dict.get(mapped_crop, 0)
                # print(crop_SAI, crop_proportion, crop_SAI * crop_proportion, SAI)
                SAI += crop_SAI * crop_proportion
            
            SAIs.append(SAI)
            assert round(total_pro) == 1
        else:
            SAIs.append(0)

    # print(SAIs)
    df['SAI'] = SAIs
    
    return df


# mapped_area_dict = S_CropType_Dict['2017']
# test_df = calculate_SAI(CHIRPS_SMAP_GFSAD_df[27184:27185], mapped_area_dict, Mapped_SAI_Dict, crop_mapping)


#### Convert Monthly N Requirement for each grid to Daily

## SAI Combined

In [13]:
Lower_lat_Lower_lon = []
Lower_lat_Upper_lon = []
Upper_lat_Lower_lon = []
Upper_lat_Upper_lon = []
F_dir = '/global/scratch/users/liuwenjin021011/data/ThesisFactorC/'
for file in os.listdir(F_dir):
    if 'Year2019_Lower_lats_Lower_lons' in file:
        Lower_lat_Lower_lon += [F_dir + file]
        
    if 'Year2019_Lower_lats_Upper_lons' in file:
        Lower_lat_Upper_lon += [F_dir + file]
        
    if 'Year2019_Upper_lats_Lower_lons' in file:
        Upper_lat_Lower_lon += [F_dir + file]
    
    if 'Year2019_Upper_lats_Upper_lons' in file:
        Upper_lat_Upper_lon += [F_dir + file]
        
Lower_lat_Lower_lon.sort()
Lower_lat_Upper_lon.sort()
Upper_lat_Lower_lon.sort()
Upper_lat_Upper_lon.sort()

In [14]:
def merge_file(file_list, save_name):
    all_df = pd.DataFrame()
    for file in file_list:
        chunk_df = pd.read_csv(file)
        all_df = pd.concat([all_df, chunk_df])
        # print(file)
    # all_df = all_df.drop(columns = ['Unnamed: 0'])
    all_df = all_df.drop_duplicates()
    all_df.to_csv(f'/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_{save_name}.csv')

    
merge_file(Lower_lat_Lower_lon, 'Lower_lat_Lower_lon')
merge_file(Lower_lat_Upper_lon, 'Lower_lat_Upper_lon')
merge_file(Upper_lat_Lower_lon, 'Upper_lat_Lower_lon')
merge_file(Upper_lat_Upper_lon, 'Upper_lat_Upper_lon')

## SAI Drop Duplicates

In [15]:
ll_df = pd.read_csv('/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_Lower_lat_Lower_lon.csv')

In [16]:
lu_df = pd.read_csv('/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_Lower_lat_Upper_lon.csv')

In [17]:
ul_df = pd.read_csv('/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_Upper_lat_Lower_lon.csv')

In [18]:
uu_df = pd.read_csv('/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_Upper_lat_Upper_lon.csv')

In [19]:
df = pd.concat([ll_df, lu_df, ul_df, uu_df])

In [22]:
df.to_csv('/global/scratch/users/liuwenjin021011/data/ThesisFactorC/Year2019_SAI.csv', header = True, index = False)